In [ ]:
import os
import torch
import faiss
import logging
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from Libraries import A0_Widgets, A1_Define
from Libraries import C1_CreateSchema, C2_Embedding, C3_CheckConstruct

In [ ]:
widgets_list = A0_Widgets.create_name_form()

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
force_download = True

In [ ]:
config = A1_Define.WidgetValues(widgets_list)

data_foler = config["data_folder"]
dcmt_path = config["dcmt_path"]
base_folder = config["base_folder"]
base_path = config["base_path"]
chunks_base = config["chunks_base"]
json_file_path = config["json_file_path"]
schema_ex_path = config["schema_ex_path"]
embedding_path = config["embedding_path"]
torch_path = config["torch_path"]
faiss_path = config["faiss_path"]
mapping_path = config["mapping_path"]
mapping_data = config["mapping_data"]

FILE_TYPE = config["FILE_TYPE"]
DATA_KEY = config["DATA_KEY"]
EMBE_KEY = config["EMBE_KEY"]
SWITCH = config["SWITCH"]
EMBEDD_MODEL = config["EMBEDD_MODEL"]
SEARCH_EGINE = config["SEARCH_EGINE"]
RERANK_MODEL = config["RERANK_MODEL"]
RESPON_MODEL = config["RESPON_MODEL"]
MERGE = config["MERGE"]
API_KEY = config["API_KEY"]

WORD_LIMIT = config["WORD_LIMIT"]
LEVEL_INPUT = config["LEVEL_INPUT"]
LEVEL_VALUES = config["LEVEL_VALUES"]

SEARCH_ENGINE = faiss.IndexFlatIP

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if (SWITCH == "Auto Model"):
    try:
        tokenizer = AutoTokenizer.from_pretrained(EMBEDD_MODEL, force_download=force_download)
        model = AutoModel.from_pretrained(EMBEDD_MODEL, force_download=force_download)
        model = model.to(device)
        print("Model and tokenizer loaded successfully")
    except Exception as e:
        raise
elif (SWITCH == "Sentence Transformer"):
    try:
        # model = SentenceTransformer(EMBEDD_MODEL).to(device)
        model = SentenceTransformer("../../cached_model")
        print("SentenceTransformer loaded successfully")
    except Exception as e:
        raise

print(f"Using: {device}")

In [ ]:
C1_CreateSchema.create_schema(json_file_path, schema_ex_path)

In [ ]:
C2_Embedding.json_embeddings(MERGE, json_file_path, torch_path, schema_ex_path, model, device, DATA_KEY, EMBE_KEY)

In [ ]:
C3_CheckConstruct.print_json(DATA_KEY, torch_path)

In [ ]:
D0_FaissConvert.convert_pt_to_faiss(
    torch_path=torch_path, 
    faiss_path=faiss_path, 
    mapping_path=mapping_path, 
    mapping_data=mapping_data, 
    data_key = DATA_KEY,
    nlist = 100, 
    use_pickle = False)